In [10]:
import pandas as pd
import numpy as np
import matplotlib 

In [11]:
global notas
filmes = pd.read_csv("./dataset/movies.csv")
filmes.columns = ["idF","nome","generos"]
notas = pd.read_csv("./dataset/ratings.csv")
notas.columns = ["idU","idF","nota","data"]
nUsuarios = notas.groupby("idU").count()
nUsuarios = len(nUsuarios)

Primeira Heuristica:
considerando que não conhecemos nada do usuario, simplesmente recomendaria os 5 filmes mais avaliados.

Quantidade de Avaliações não necessariamente quer dizer um filme excelente...

In [12]:
qtdAvaliacoes = notas.groupby("idF").count()
qtdAvaliacoes.columns = ["qtdAvaliacoes","d1","d2"]
qtdAvaliacoes.drop('d1', axis='columns', inplace=True)
qtdAvaliacoes.drop('d2', axis='columns', inplace=True)
filmes = filmes.set_index("idF").join(qtdAvaliacoes)

Segunda Heuristica:
5 filmes mais bem avaliados filtrando filmes com mais de 60 avaliações.

Filmes de nicho iam ser muito bem ranqueados...

In [15]:
notaMedia = notas.groupby("idF").mean()
notaMedia.columns = ["d1","notaMedia","d2"]
notaMedia.drop('d1', axis='columns', inplace=True)
notaMedia.drop('d2', axis='columns', inplace=True)
filmes = filmes.join(notaMedia)

In [16]:
filmesComMaisDe60Ava = filmes.query("qtdAvaliacoes > 60").sort_values("notaMedia", ascending=False).head(10)

In [17]:
filmesComMaisDe60Ava

,nome,generos,qtdAvaliacoes,notaMedia
idF,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000
912,Casablanca (1942),Drama|Romance,100.0,4.240000


Terceira Heuristica: filmes com o mesmo genero do ultimo que o usuario assistiu, que tenham mais de 60 avaliações.

In [69]:
#gerar lista aleatória de 10 filmes
assistidos = np.random.randint(100, size=10)
assistidos = assistidos.tolist()

In [70]:
filmes["generos"].iloc[assistidos[-1]]
generoDoUltimoFilmeAssistido = filmes["generos"].iloc[assistidos[-1]]
filmesComMaisDe60Ava.query(f"generos == '{generoDoUltimoFilmeAssistido}'").drop(assistidos, errors="ignore").sort_values("notaMedia", ascending=False).head(10)

,nome,generos,qtdAvaliacoes,notaMedia
idF,,,,
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905


Quarta Heuristica: filmes que o usuario mais proximo de você assistiu utilizando o KNN 

In [71]:
def notasDoUsuario(id):
    notasa = notas.query(f"idU=={id}")[["idF","nota"]].set_index("idF")
    return notasa

In [72]:
def distancia(concat):
    notaP1 = np.array(concat["notaP1"])
    notaP2 = np.array(concat["notaP2"])
    retorno = np.linalg.norm(notaP1 - notaP2)
    return retorno

In [73]:
def distanciaDeTodos(usuarioTestado,numeroDeObjetos = nUsuarios, minimo = 5):
    notas1 = notasDoUsuario(usuarioTestado)
    dist = []
    for idU in notas["idU"].unique()[:numeroDeObjetos]:
        notas2 = notasDoUsuario(idU)
        print(idU)
        concatenado = notas1.merge(notas2, left_on="idF", right_on="idF", suffixes=("P1", "P2"))
        if(len(concatenado) >= minimo):
            dist.append([usuarioTestado,idU,distancia(concatenado)])
    dist = pd.DataFrame(dist,columns=["usuarioTestado","outroPessoa","Distancia"])
    dropar = dist.loc[dist["outroPessoa"] == usuarioTestado].index
    dist.drop(dropar, errors="ignore", inplace=True)
    dist.sort_values("Distancia", inplace=True)
    dist.reset_index(inplace=True)
    dist.drop('index', axis='columns', inplace=True)
    return dist

In [ ]:
global dist
dist = distanciaDeTodos(1)

In [75]:
def filmesNaoVistosUsuarioMaisProximo(dist):
    notasTestado = notasDoUsuario(dist.iloc[0]["usuarioTestado"])
    notasUsuarioMaisProximo = notasDoUsuario(dist.iloc[0]["outroPessoa"])
    filmesNaoVisto = notasUsuarioMaisProximo.drop(notasTestado.index, errors="ignore")
    filmesNaoVisto.sort_values("nota", ascending=False, inplace=True)
    return filmesNaoVisto

In [76]:
filmesNaoVistos = filmesNaoVistosUsuarioMaisProximo(dist)

In [77]:
filmesNaoVistos.head()

,nota
idF,
8636,5.0
58559,5.0
33794,5.0
4993,5.0
5349,5.0


Quinta Heuristica: Sugerindo baseado em varios usuarios com o KNN

In [78]:
#definir quantidade de usuarios
#dist.sort_values("Distancia", ascending=True)[:5]
dist.sort_values("Distancia", ascending=True).set_index("outroPessoa")

,usuarioTestado,Distancia
outroPessoa,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
9,1,1.000000
49,1,1.000000
...,...,...
474,1,18.594354
160,1,18.794946
217,1,19.646883


In [107]:
def usuariosMaisProximos(idsUsuarios,qtdUsuarios = len(dist)):
    usuariosMaisProximos = dist.sort_values("Distancia", ascending=True).set_index("outroPessoa")[:qtdUsuarios]
    notasTestado = []
    for index,row in usuariosMaisProximos.iterrows():
        notas = notasDoUsuario(index)
        for index,row in notas.iterrows(): 
            notasTestado.append(row)
    notasTestado = pd.DataFrame(notasTestado, columns = ["nota"])
    notasTestado.index.name = "idF"
    return notasTestado

In [108]:
maisProx = usuariosMaisProximos(1,10)

In [110]:
maisProx.groupby(maisProx.index).mean().join()

,nota
idF,
2,4.500000
32,5.000000
41,3.000000
110,4.166667
187,3.000000
...,...
139385,4.666667
150548,5.000000
152081,4.500000
